# Search Tester Analysis
*Follow these steps to compare legacy results to the new api results.*

### Setup the environment

In [1]:
%run ./local/setup.ipynb

Unable to configure logging, attempted conf:/home/notebook/.local/lib/python3.10/site-packages/ppr_api/logging.conf


In [2]:
query = db.session.execute("select category, synonyms_text, stems_text from synonym")

In [3]:
query

In [3]:
synonyms = query.fetchall()

In [4]:
synonym_list = []
for syns in synonyms:
    words1 = syns[1].split(',')
    words2 = syns[2].split(',')
    for word in words2:
        if word not in words1:
            words1.append(word)
    synonym_list.append(words1)
    


In [10]:
synonym_list

[['davine',
  ' devine',
  ' devyne',
  ' divine',
  ' divyne',
  'davin',
  ' devin',
  ' devyn',
  ' divin',
  ' divyn']]

In [6]:
import csv

with open('synonym.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    count = 0
    for synonyms in synonym_list:
        for synonym in synonyms:
            spamwriter.writerow([count, synonym])
        count += 1

### Find search batches (choose the data set)
*find the ones you want to analyze*

In [2]:
search_type = None
after_date = None
before_date = None
### OPTIONAL: refine your batch search with any of the following
# search_type = SearchRequest.SearchTypes.XXXX
# before_date = _datetime_in_utc_
after_date = "2022-01-21 18:48:30.936128"
batches = TestSearchBatch.find_search_batches(search_type, after_date, before_date)
for batch in batches:
    print(batch.id, batch.test_date, batch.search_type, batch.sim_val_business, batch.sim_val_first_name, batch.sim_val_last_name)


804 2022-01-21 19:53:31.904331 BS 0.6 0.4 0.29
806 2022-01-21 21:13:12.357598 BS 0.6 0.4 0.29
807 2022-01-21 21:14:32.023847 BS 0.6 0.4 0.29


In [3]:
# add the ids of the batches you want to analyze
batch_ids = []
for batch in batches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. batch.id > _ , batch.sim_val_business > _ ...
        batch_ids.append(batch.id)

### Get the batch json

In [4]:
batches_to_analyze = []
for batch_id in batch_ids:
    batches_to_analyze.append(TestSearchBatch.find_by_id(batch_id))

# Analysis

### Exact Matches

#### Pass rate

In [5]:
searches_total = 0
searches_passed = 0
failed_searches = []
all_searches = []
for batch in batches_to_analyze:
    for search in batch.searches:
        searches_total += 1
        all_searches.append(search)
        if len(search.missed_matches(TestSearchResult.MatchType.EXACT.value)) > 0:
            failed_searches.append(search)
        else:
            searches_passed += 1
print('pass rate: ', searches_passed/searches_total)
print('number of failed searches: ', len(failed_searches))

pass rate:  1.0
number of failed searches:  0


#### Failed Searches

*select failed searches to analyze*

In [6]:
for search in failed_searches:
    json = search.json
    print('###########################################################')
    print('criteria: ', json['criteria'])
    print('total expected: ', len(json['matchesExact']['resultsLegacy']), 'missed: ', len(json['matchesExact']['missedMatches']))

In [ ]:
### print out json for specific search
# print(failed_searches[0].json)

In [7]:
exact_searches_analysis = []
for search in failed_searches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. search.search_criteria == _, len(search.missed_matches(TestSearchResult.MatchType.EXACT.value)) > _
        exact_searches_analysis.append(search)

*missed matches*

In [8]:
import re
### manually iterate through exact_searches_analysis one by one to see the missed matches
# search = exact_searches_analysis[1]
# for match in search.missed_matches(TestSearchResult.MatchType.EXACT.value):
#     match['details'] = re.sub(' +', ' ', match['details'])
#     print('-------------------------------------------------------------------------------')
#     print('result:', match['details'])
#     print('reg num:', match['documentId'])
#     print('index: ', match['index'])
### print out all of them
for search in exact_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('Missed Matches')
    for match in search.missed_matches(TestSearchResult.MatchType.EXACT.value):
        print('-------------------------------------------------------------------------------')
        print('result:', match['details'])
        print('reg num:', match['documentId'])
        print('index: ', match['index'])

*results diff*

In [9]:
import re
### manually iterate through exact_searches_analysis one by one
# search = exact_searches_analysis[1]
# print('criteria: ', search.search_criteria)
# print('-------------------------------------------------------------')
# print('  legacy                           api')
# print('-------------------------------------------------------------')
# legacy_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.LEGACY.value)
# api_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.API.value)
# length = max(len(legacy_results), len(api_results))
# for i in range(length):
#     if i < len(legacy_results) and i < len(api_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#     elif i < len(legacy_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#     elif i < len(api_results):
#         print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### print out all of them
for search in exact_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('-------------------------------------------------------------')
    print('  legacy                           api')
    print('-------------------------------------------------------------')
    legacy_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.LEGACY.value)
    api_results = search.get_results(TestSearchResult.MatchType.EXACT.value, TestSearchResult.Source.API.value)
    length = max(len(legacy_results), len(api_results))
    for i in range(length):
        if i < len(legacy_results) and i < len(api_results):
            legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
            print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
        elif i < len(legacy_results):
            legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
            print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
        elif i < len(api_results):
            print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')


### Similar Matches

#### Pass rate

In [10]:
searches_total = 0
searches_passed = 0
failed_searches = []
for batch in batches_to_analyze:
    for search in batch.searches:
        searches_total += 1
        # fails if missed matches
        if len(search.missed_matches(TestSearchResult.MatchType.SIMILAR.value)) > 0:
            failed_searches.append(search)
        # fails if order is off
        elif search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value) != 0:
            failed_searches.append(search)
        else:
            searches_passed += 1
print('pass rate: ', searches_passed/searches_total)
print('number of failed searches: ', len(failed_searches))

pass rate:  0.673728813559322
number of failed searches:  154


#### Failed searches

*select failed searches to analyze*

In [11]:
for search in failed_searches:
    json = search.json
    print('###########################################################')
    print('criteria: ', json['criteria'])
    print('total expected: ', len(json['matchesSimilar']['resultsLegacy']), ' missed: ', len(json['matchesSimilar']['missedMatches']))
    print('first fail index: ', json['matchesSimilar']['firstFailIndex'])
    print('avg index diff: ', json['matchesSimilar']['avgIndexDiff'])

###########################################################
criteria:  DR. CHRISTOPHER RICKARDS INC                                          
total expected:  6  missed:  6
first fail index:  -1
avg index diff:  0
###########################################################
criteria:  GREENER SPACES                                                        
total expected:  20  missed:  0
first fail index:  0
avg index diff:  10.6
###########################################################
criteria:  DIG IT CONTRACTING                                                    
total expected:  22  missed:  22
first fail index:  0
avg index diff:  0
###########################################################
criteria:  C & D CAPITAL                                                         
total expected:  3  missed:  3
first fail index:  0
avg index diff:  0
###########################################################
criteria:  JB OFFICE SYSTEMS                                                     

###########################################################
criteria:  J.ROSE CORPORATION                                                    
total expected:  2  missed:  2
first fail index:  -1
avg index diff:  0
###########################################################
criteria:  THE OWNERS STRATA PLAN EPS 6752                                       
total expected:  10  missed:  1
first fail index:  0
avg index diff:  31.5
###########################################################
criteria:  DREAM VILLA                                                           
total expected:  1  missed:  0
first fail index:  0
avg index diff:  1.0
###########################################################
criteria:  KALIA HOLDINGS LTD                                                    
total expected:  13  missed:  13
first fail index:  0
avg index diff:  0
###########################################################
criteria:  DR. SUMEET KALIA INC.                                               

###########################################################
criteria:  STRAIT PIPELINE & INTEGRITY LTD.                                      
total expected:  1  missed:  0
first fail index:  0
avg index diff:  72.0
###########################################################
criteria:  PIZZA PIZZA LIMITED                                                   
total expected:  2  missed:  2
first fail index:  0
avg index diff:  0
###########################################################
criteria:  CARLETON GROUP INC.                                                   
total expected:  1  missed:  1
first fail index:  0
avg index diff:  0
###########################################################
criteria:  NEXT LEVEL CHILDCARE VALLEY FAIR INC.                                 
total expected:  6  missed:  6
first fail index:  -1
avg index diff:  0
###########################################################
criteria:  MISSION GROUP INC.                                                    
to

###########################################################
criteria:  WHITE DRYWALL & DEVELOPMENT LTD                                       
total expected:  4  missed:  3
first fail index:  0
avg index diff:  2.0
###########################################################
criteria:  APNA DRYWALL & CONSTRUCTION LTD                                       
total expected:  1  missed:  1
first fail index:  0
avg index diff:  0
###########################################################
criteria:  RYAN COMPANY LIMITED                                                  
total expected:  3  missed:  3
first fail index:  0
avg index diff:  0
###########################################################
criteria:  NORTH SHORE BETTER                                                    
total expected:  1  missed:  1
first fail index:  -1
avg index diff:  0
###########################################################
criteria:  LOON PROPERTIES INC.                                                  
tot

In [ ]:
### print out json for specific search
# print(failed_searches[0].json)

In [12]:
similar_searches_analysis = []
for search in failed_searches:
    # fill in the clause (if you want all of them then set to True or remove)
    if True:  # i.e. search.search_criteria == _, len(search.missed_matches(TestSearchResult.MatchType.SIMILAR.value)) > _
        similar_searches_analysis.append(search)

*first failed indexes*

In [11]:
### manually iterate through
search = similar_searches_analysis[0]
print(search.fail_index(TestSearchResult.MatchType.SIMILAR.value))

### print out all of them
# for search in similar_searches_analysis:
#     print(search.fail_index(TestSearchResult.MatchType.SIMILAR.value))

0


In [7]:
### avgs
total_no_fails = 0
total_fail_index = 0
for search in similar_searches_analysis:
    fail_index = search.fail_index(TestSearchResult.MatchType.SIMILAR.value)
    if fail_index == -1:
        total_no_fails += 1
    else:
        total_fail_index += fail_index

num_searches_failed = len(similar_searches_analysis) - total_no_fails
print('Number of searches with fail indexes: ', num_searches_failed)
print('avg fail index: ', total_fail_index/num_searches_failed)

Number of searches with fail indexes:  608
avg fail index:  0.32894736842105265


*avg order difference between legacy and api results (does NOT include missed matches)*

In [8]:
### manually iterate through
search = similar_searches_analysis[0]
print(search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value))

### print out all of them
# for search in similar_searches_analysis:
#     print(search.avg_index_diff(TestSearchResult.MatchType.SIMILAR.value))

0


*missed matches*

In [13]:
import re
### manually iterate through
# search = similar_searches_analysis[0]
# print('criteria: ', search.search_criteria)
# print('Missed Matches')
# for match in search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
#     match['details'] = re.sub(' +', ' ', match['details'])
#     print('-------------------------------------------------------------------------------')
#     print('result:', match['details'])
#     print('reg num:', match['documentId'])
#     print('index: ', match['index'])

### print out all of them
for search in similar_searches_analysis:
    print('##################################################################################')
    print('criteria: ', search.search_criteria)
    print('Missed Matches')
    for match in search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
        print('-------------------------------------------------------------------------------')
        print('result:', match['details'])
        print('reg num:', match['documentId'])
        print('index: ', match['index'])

##################################################################################
criteria:  DR. CHRISTOPHER RICKARDS INC                                          
Missed Matches
-------------------------------------------------------------------------------
result: DR. CHRISTOPHER R. CUNNINGHAM INC.                             
reg num: 6678098
index:  0
-------------------------------------------------------------------------------
result: DR. CHRISTOPHER R. CUNNINGHAM INC.                             
reg num: 623069I
index:  1
-------------------------------------------------------------------------------
result: DR. CHRISTOPHER R. OLYNIK INC.                                 
reg num: 729819H
index:  2
-------------------------------------------------------------------------------
result: DR. CHRISTOPHER R. OLYNIK INC.                                 
reg num: 385987L
index:  3
-------------------------------------------------------------------------------
result: DR. CHRISTOPHER 

-------------------------------------------------------------------------------
result: WEST COAST RETINA CONSULTANTS INC.                             
reg num: 677016I
index:  0
##################################################################################
criteria:  MCLOUD TECHNOLOGIES SERVICES                                          
Missed Matches
##################################################################################
criteria:  SILVER ARROW CARS                                                     
Missed Matches
-------------------------------------------------------------------------------
result: SILVER ARROW CARS LTD                                          
reg num: 395096M
index:  0
-------------------------------------------------------------------------------
result: SILVER ARROW CARS LTD                                          
reg num: 569417M
index:  1
-------------------------------------------------------------------------------
result: SILVER ARROW CA

-------------------------------------------------------------------------------
result: TRIDEV DEVELOPMENTS LTD.                                       
reg num: 985280D
index:  1
##################################################################################
criteria:  REGULUS RESOURCES INC                                                 
Missed Matches
-------------------------------------------------------------------------------
result: REGAL RESOURCES INC                                            
reg num: 541368M
index:  0
##################################################################################
criteria:  MCMILLAN LLP                                                          
Missed Matches
-------------------------------------------------------------------------------
result: MCMILLAN DUBO LLP                                              
reg num: 879176J
index:  0
-------------------------------------------------------------------------------
result: MCMILLAN DUBO L

-------------------------------------------------------------------------------
result: C D M HOLDINGS LTD                                             
reg num: 4850937
index:  0
-------------------------------------------------------------------------------
result: KDMS HOLDINGS LTD                                              
reg num: 522203K
index:  1
-------------------------------------------------------------------------------
result: KDMS HOLDINGS LTD.                                             
reg num: 522203K
index:  2
-------------------------------------------------------------------------------
result: KDMS HOLDINGS LTD.                                             
reg num: 522203K
index:  3
-------------------------------------------------------------------------------
result: KDMS HOLDINGS LTD.                                             
reg num: 416387L
index:  4
-------------------------------------------------------------------------------
result: KDMS HOLDINGS LTD

-------------------------------------------------------------------------------
result: NORLAND FOREST PRODUCTS LTD                                    
reg num: 443545E
index:  1
-------------------------------------------------------------------------------
result: NORLAND FOREST PRODUCTS LTD.                                   
reg num: 075317E
index:  2
-------------------------------------------------------------------------------
result: NORLAND FOREST PRODUCTS LTD.                                   
reg num: 237348G
index:  3
-------------------------------------------------------------------------------
result: NORLAND FOREST PRODUCTS LTD.                                   
reg num: 322591G
index:  4
-------------------------------------------------------------------------------
result: NORLAND LIMITED                                                
reg num: 720436L
index:  5
-------------------------------------------------------------------------------
result: NORLAND LIMITED  

...

-------------------------------------------------------------------------------
result: AUTOPRO AUTOMATION CONSULTANTS LTD                             
reg num: 675077K
index:  0
-------------------------------------------------------------------------------
result: AUTOPRO AUTOMATION CONSULTANTS LTD                             
reg num: 684606L
index:  1
-------------------------------------------------------------------------------
result: AUTOPRO AUTOMATION CONSULTANTS LTD.                            
reg num: 222695L
index:  2
-------------------------------------------------------------------------------
result: AUTOPRO AUTOMATION CONSULTANTS LTD.                            
reg num: 823166L
index:  5
##################################################################################
criteria:  MCLOUD TECHNOLOGIES SERVICES                                          
Missed Matches
##################################################################################
criteria:  KA3 HOLDIN

##################################################################################
criteria:  QUANTUM HELICOPTERS LTD.                                              
Missed Matches
-------------------------------------------------------------------------------
result: QUANTUM HELICOPTERS LTD., AS LESSEE                            
reg num: 650375D
index:  0
-------------------------------------------------------------------------------
result: QUANTUM HELICOPTERS LTD., AS LESSEE                            
reg num: 650375D
index:  1
##################################################################################
criteria:  GRITTEN HOLDINGS LTD                                                  
Missed Matches
-------------------------------------------------------------------------------
result: GRATTON HOLDINGS (1984) LTD                                    
reg num: 4515323
index:  0
-------------------------------------------------------------------------------
result: GRATTON HOLDING

-------------------------------------------------------------------------------
result: LANE PROPERTY ADVISORS INC                                     
reg num: 7272268
index:  0
-------------------------------------------------------------------------------
result: LOM PROPERTIES INC.                                            
reg num: 254895G
index:  1
-------------------------------------------------------------------------------
result: LYON PROPERTIES INC.                                           
reg num: 803930J
index:  20
-------------------------------------------------------------------------------
result: LYON PROPERTIES INC.                                           
reg num: 803937J
index:  21
##################################################################################
criteria:  HOPE FOR FREEDOM                                                      
Missed Matches
##################################################################################
criteria:  FRASER S

-------------------------------------------------------------------------------
result: P. & C. HOLDINGS LTD.                                          
reg num: 9555150
index:  0
-------------------------------------------------------------------------------
result: PCI HOLDINGS CORP                                              
reg num: 091986J
index:  1
-------------------------------------------------------------------------------
result: PCI HOLDINGS CORP                                              
reg num: 491762K
index:  2
-------------------------------------------------------------------------------
result: PCI HOLDINGS CORP                                              
reg num: 491764K
index:  3
-------------------------------------------------------------------------------
result: PCI HOLDINGS CORP                                              
reg num: 076282L
index:  4
-------------------------------------------------------------------------------
result: PCI HOLDINGS CORP

##################################################################################
criteria:  ANDALCO INVESTMENTS INC.                                              
Missed Matches
-------------------------------------------------------------------------------
result: ANDAL INVESTMENTS LTD.                                         
reg num: 159081E
index:  0
##################################################################################
criteria:  CAPREIT LIMITED PARTNERSHIP                                           
Missed Matches
##################################################################################
criteria:  CAPREIT LIMITED PARTNERSHIP                                           
Missed Matches
##################################################################################
criteria:  CAPREIT LIMITED PARTNERSHIP                                           
Missed Matches
##################################################################################
criteria:  METAOPT

*results diff*

In [14]:
missed = 0
for search in similar_searches_analysis:
    if search.missed_matches(TestSearchResult.MatchType.SIMILAR.value):
        missed += 1

print('searches with missed matches: ', missed)
print('percent of total that do not have a missed similar: ', (searches_total-missed)/searches_total)

print('total searches: ', searches_total)

# import re
# ### manually iterate through similar_searches_analysis one by one
# search = similar_searches_analysis[0]
# print('criteria: ', search.search_criteria)
# print('-------------------------------------------------------------')
# print('  legacy                           api')
# print('-------------------------------------------------------------')
# legacy_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.LEGACY.value)
# api_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.API.value)
# length = max(len(legacy_results), len(api_results))
# for i in range(length):
#     if i < len(legacy_results) and i < len(api_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#     elif i < len(legacy_results):
#         legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#         print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#     elif i < len(api_results):
#         print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

### print out all of them
# for search in similar_searches_analysis:
#     print('criteria: ', search.search_criteria)
#     print('-------------------------------------------------------------')
#     print('  legacy                           api')
#     print('-------------------------------------------------------------')
#     legacy_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.LEGACY.value)
#     api_results = search.get_results(TestSearchResult.MatchType.SIMILAR.value, TestSearchResult.Source.API.value)
#     length = max(len(legacy_results), len(api_results))
#     for i in range(length):
#         if i < len(legacy_results) and i < len(api_results):
#             legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#             print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} | {api_results[i]["documentId"]} {api_results[i]["details"]}')
#         elif i < len(legacy_results):
#             legacy_results[i]['details'] = re.sub(' +', ' ', legacy_results[i]['details'])
#             print(f'{i}: {legacy_results[i]["documentId"]} {legacy_results[i]["details"]} |')
#         elif i < len(api_results):
#             print(f'{i}:                    | {api_results[i]["documentId"]} {api_results[i]["details"]}')

searches with missed matches:  113
percent of total that have a missed similar:  0.760593220338983
total searches:  472


### Create MD file of notebook run
**NOTE:** save notebook (i.e. _cmd s_) now to have results show in markdown file

In [17]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [18]:
md_name = nb_name[:-6]+'.md'

In [ ]:
%%bash -s "$nb_name" "$md_name"
jupyter nbconvert $1 --to markdown --output $2